In [110]:
# %load data_clean.py
import os
import shutil
import zipfile

import pandas as pd

In [ ]:
# 解壓縮檔案
def unzip_files(source_dir, dest_dir):
    for item in os.listdir(source_dir): # loop through items in dir
        if item.endswith('.zip'): # check for ".zip" extension
            file_name = source_dir + "/" + item # get full path of files
            zip_ref = zipfile.ZipFile(file_name) # create zipfile object
            zip_ref.extractall(dest_dir) # extract file to dir
            zip_ref.close() # close file
            # os.remove(file_name) # delete zipped file
        else:
            continue
# 解壓縮所有的資料並移動到./data
# 選定zip資料夾
source_dir = './option11002-11004'
dest_dir = './data/raw_data'
unzip_files(source_dir, dest_dir)

In [129]:
all_data_df = pd.read_csv('all_data.csv', encoding='utf-8-sig')



In [130]:
# 只留下MTD_PROD_ID含有TX的台指期選擇權
all_data_df = all_data_df[all_data_df['MTF_PROD_ID'].str.contains('TXO')]

# 刪除'MTF_PROD_ID'欄位中的空格
all_data_df['MTF_PROD_ID'] = all_data_df['MTF_PROD_ID'].str.replace(' ', '')

In [131]:
# 留下單式商品
all_data_df = all_data_df[all_data_df['MTF_SC_CODE']=='S']
all_data_df

,MTF_DATE,MTF_PROD_ID,MTF_PROD_ID1,MTF_BS_CODE1,MTF_M_PRICE1,MTF_M_QNTY1,MTF_PROD_ID2,MTF_BS_CODE2,MTF_M_PRICE2,MTF_M_QNTY2,MTF_BS_CODE,MTF_PRICE,MTF_QNTY,MTF_OC_CODE,MTF_M_INST,MTF_ORIG_TIME,MTF_SC_CODE,FCM_NO+ORDER_NO+,OQ_CODE
0,20210311,TXO16500C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,40.0,10,A,17797259,13:33:26.688000,S,5D45C3D99531733E199C5D25858A8F09,NaN
1,20210311,TXO16500C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,40.0,10,0,17797259,13:33:26.688000,S,FD2F017B61183A8F935E9647EB30637F,NaN
2,20210311,TXO16300C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,96.0,5,1,17797285,13:33:26.826000,S,E4942309B8CA5FF0E719466371055AAA,NaN
3,20210311,TXO16300C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,96.0,4,1,17797285,13:33:26.826000,S,14A0DC877A14D4D4A782DF85186DCAD6,NaN
4,20210311,TXO16300C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,96.0,1,0,17797285,13:33:26.826000,S,BE8CCFF772750D6582B0F1EB439ED466,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19130170,20210326,TXO16300P1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,300.0,1,9,9260568,10:58:57.777000,S,7EA337ECEB04E28A9E5A272C60078E80,NaN
19130171,20210326,TXO16300P1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,300.0,1,1,9260568,10:58:57.777000,S,19FDDC937370ECF53813EC2FE118B0E3,NaN
19130172,20210326,TXO15700P1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,111.0,1,0,9260719,10:58:58.224000,S,89F3A0B23E08398BF8211A946DC63645,NaN
19130173,20210326,TXO15700P1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,111.0,1,0,9260719,10:58:58.224000,S,5F2C1A49FE5A6D125B4D4E273B7BB781,NaN


In [132]:
# 新增履約價格欄位
all_data_df['strike_price'] = all_data_df['MTF_PROD_ID'].str.slice(3, 8)
all_data_df['strike_price'] = all_data_df['strike_price'].astype(int)
all_data_df['strike_price']

0           16500
1           16500
2           16300
3           16300
4           16300
            ...  
19130170    16300
19130171    16300
19130172    15700
19130173    15700
19130184    15000
Name: strike_price, Length: 6550544, dtype: int64

In [133]:
# 新增履約月份欄位
all_data_df['month_alpha'] = all_data_df['MTF_PROD_ID'].str.slice(8, 9)
all_data_df['month_alpha']

# 月份映射Ａ~L(112月)買權,M~X(112月)賣權
alpha_to_month = {
    'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6,
    'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12,
    'M': 1, 'N': 2, 'O': 3, 'P': 4, 'Q': 5, 'R': 6,
    'S': 7, 'T': 8, 'U': 9, 'V': 10, 'W': 11, 'X': 12
}

# 轉換month_alpha
all_data_df['month'] = all_data_df['month_alpha'].map(alpha_to_month)

# 轉換month為字串
all_data_df['month'] = all_data_df['month'].astype(int).astype(str)


In [134]:
# 將MTF_DATE和MTF_ORIG_TIME欄位的數據轉換為字串格式
all_data_df['MTF_DATE'] = all_data_df['MTF_DATE'].astype(str)
all_data_df['MTF_ORIG_TIME'] = all_data_df['MTF_ORIG_TIME'].astype(str)

# 合併MTF_DATE和MTF_ORIG_TIME欄位的數據
all_data_df['datetime'] = all_data_df['MTF_DATE'] + ' ' + all_data_df['MTF_ORIG_TIME']

# 將datetime欄位的數據轉換為日期時間格式
all_data_df['datetime'] = pd.to_datetime(all_data_df['datetime'], format='%Y%m%d %H:%M:%S.%f')
all_data_df['datetime']

0          2021-03-11 13:33:26.688
1          2021-03-11 13:33:26.688
2          2021-03-11 13:33:26.826
3          2021-03-11 13:33:26.826
4          2021-03-11 13:33:26.826
                     ...          
19130170   2021-03-26 10:58:57.777
19130171   2021-03-26 10:58:57.777
19130172   2021-03-26 10:58:58.224
19130173   2021-03-26 10:58:58.224
19130184   2021-03-26 10:55:15.382
Name: datetime, Length: 6550544, dtype: datetime64[ns]

In [135]:
# 選擇要保留的欄位
all_data_df = all_data_df[['MTF_DATE', 'MTF_PROD_ID', 'MTF_BS_CODE', 'MTF_PRICE', 'MTF_QNTY', 'MTF_OC_CODE', 'strike_price', 'month_alpha', 'month', 'datetime']]

In [136]:
# MTF_PROD_ID分群
prod_df = all_data_df.groupby('month_alpha')

# 列出所有的分群
group_names = prod_df.groups.keys()
print(group_names)

dict_keys(['B', 'C', 'D', 'E', 'F', 'G', 'I', 'L', 'N', 'O', 'P', 'Q', 'R', 'S', 'U', 'X'])


In [137]:
import re

# 輸出每一組的數據到一個csv檔案
for name, group in prod_df:
    month = group['month'].iloc[0]
    MTF_BS_CODE = group['MTF_BS_CODE'].iloc[0]
    cleaned_name = re.sub('[\W_]+', '', name)
    filename = f'./data/group/{month}_{MTF_BS_CODE}.csv'
    group.to_csv(filename, index=False, encoding='utf-8-sig')